# Import packages

In [1]:
%pip install -q pandas matplotlib numpy scikit-learn 
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [3]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0))

CUDA Available: True
GPU Name: NVIDIA GeForce RTX 2070 with Max-Q Design


# Import dataset

In [61]:
# Load the final_df dataset
final_df = pd.read_csv('../final_df.csv')
final_df

,codmpio,year,tc_loss,gandina,gcaribe,gpacifica,gorinoquia,gamazonia,areaoficialhm2,discapital,...,subnational1_VALLE DEL CAUCA,subnational1_VAUPES,subnational1_VICHADA,cluster_kmeans_0,cluster_kmeans_1,cluster_kmeans_2,cluster_kmeans_3,cluster_kmeans_4,cluster_kmeans_5,cluster_kmeans_6
0,5873.0,2003.0,2.000,1.0,0.0,0.0,0.0,0.0,180100.0,122.0,...,0,0,0,0,0,0,0,0,1,0
1,5873.0,2004.0,18.000,1.0,0.0,0.0,0.0,0.0,180100.0,122.0,...,0,0,0,0,0,0,0,0,1,0
2,5873.0,2005.0,13.375,1.0,0.0,0.0,0.0,0.0,180100.0,122.0,...,0,0,0,0,0,0,0,0,1,0
3,5873.0,2006.0,10.875,1.0,0.0,0.0,0.0,0.0,180100.0,122.0,...,0,0,0,0,0,0,0,0,1,0
4,5873.0,2007.0,23.000,1.0,0.0,0.0,0.0,0.0,180100.0,122.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22137,88564.0,2019.0,0.000,0.0,1.0,0.0,0.0,0.0,2000.0,98.0,...,0,0,0,0,1,0,0,0,0,0
22138,88564.0,2020.0,102.000,0.0,1.0,0.0,0.0,0.0,2000.0,98.0,...,0,0,0,0,1,0,0,0,0,0
22139,88564.0,2021.0,25.000,0.0,1.0,0.0,0.0,0.0,2000.0,98.0,...,0,0,0,0,1,0,0,0,0,0
22140,88564.0,2022.0,4.125,0.0,1.0,0.0,0.0,0.0,2000.0,98.0,...,0,0,0,0,1,0,0,0,0,0


,codmpio,year,tc_loss,gandina,gcaribe,gpacifica,gorinoquia,gamazonia,areaoficialhm2,discapital,...,subnational1_VALLE DEL CAUCA,subnational1_VAUPES,subnational1_VICHADA,cluster_kmeans_0,cluster_kmeans_1,cluster_kmeans_2,cluster_kmeans_3,cluster_kmeans_4,cluster_kmeans_5,cluster_kmeans_6
2,5873.0,2003.0,2.000,1.0,0.0,0.0,0.0,0.0,180100.0,122.0,...,0,0,0,0,0,0,0,0,1,0
3,5873.0,2004.0,18.000,1.0,0.0,0.0,0.0,0.0,180100.0,122.0,...,0,0,0,0,0,0,0,0,1,0
4,5873.0,2005.0,13.375,1.0,0.0,0.0,0.0,0.0,180100.0,122.0,...,0,0,0,0,0,0,0,0,1,0
5,5873.0,2006.0,10.875,1.0,0.0,0.0,0.0,0.0,180100.0,122.0,...,0,0,0,0,0,0,0,0,1,0
6,5873.0,2007.0,23.000,1.0,0.0,0.0,0.0,0.0,180100.0,122.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24551,88564.0,2019.0,0.000,0.0,1.0,0.0,0.0,0.0,2000.0,98.0,...,0,0,0,0,1,0,0,0,0,0
24552,88564.0,2020.0,102.000,0.0,1.0,0.0,0.0,0.0,2000.0,98.0,...,0,0,0,0,1,0,0,0,0,0
24553,88564.0,2021.0,25.000,0.0,1.0,0.0,0.0,0.0,2000.0,98.0,...,0,0,0,0,1,0,0,0,0,0
24554,88564.0,2022.0,4.125,0.0,1.0,0.0,0.0,0.0,2000.0,98.0,...,0,0,0,0,1,0,0,0,0,0


# Dataset split     

In [42]:
# Step 1: Temporal split (e.g., train until 2018)
train_df = final_df[final_df['year'] <= 2018]
test_df = final_df[final_df['year'] > 2018]

# Step 2: Within training years, split municipalities
train_municipalities, test_municipalities = train_test_split(train_df['codmpio'].unique(), test_size=0.2, random_state=42)

# Apply municipality split
train_df = train_df[train_df['codmpio'].isin(train_municipalities)]
test_df = test_df[test_df['codmpio'].isin(test_municipalities)]

print(f"Final Train Set: {train_df.shape}")
print(f"Final Test Set: {test_df.shape}")


Final Train Set: (13528, 1235)
Final Test Set: (1045, 1235)
